# Amazon Nova Web Grounding with Bedrock converse and converse_stream APIs

Amazon Nova Web Grounding enhances Nova models by connecting them to real-time information beyond their knowledge cutoff, which results in more accurate and reliable responses. This feature is provided by the systemTool parameter when using the Bedrock Converse API and Invoke APIs. 

This tutorial walks through how to setup Nova Web Grounding and use it with the Amazon Bedrock converse and converse_stream APIs. 

## Step 1
- Log into your AWS account
- Go to Bedrock in your console
- Select Region - us-east-1
- Select Model Access from side tab
- Enable Nova Premier access

## Step 2: Configure AWS CLI with Default Profile

### Do this step if you don't have credentials
#### Getting Your AWS Credentials

1. Go to IAM → Users → Create User
2. Give a suitable name for the user and hit `Next`
3. Add relevant permissions to enable Bedrock access and finish setting up the user.
    * bedrock:ListFoundationModels, bedrock:InvokeModel or use AmazonBedrockFullAccess if your security policies allow
4. Once the user is created, go to Security credentials tab
5. Click -> Create access key
6. Choose Command Line Interface (CLI) and Create key
7. Copy the Access Key ID and Secret Access Key

### Configure AWS Profile
Open your choice of terminal and Run this command.
```bash
aws configure --profile nova-grounding-test-profile
```

You'll be asked for:
- AWS Access Key ID
- AWS Secret Access Key
- Default region name (enter: us-east-1)
- Default Format type (optionally you can enter: json)

If you decide to change the name of the profile then update the client below accordingly

## Step 3
Install the python SDK. Run the below code only once and that will install the required packages automatically.

In [ ]:
# We will first install Python SDK. You need to run this only one time
!pip install boto3

## Step 4: Run the below code
Below code sample will create the function which can then be called with a query. You need to only run this once unless you erase the context in which case you will have to run this again.

In [ ]:
import json
import boto3
import time
from botocore.config import Config

# Configure timeout settings for the Bedrock API client
# These settings help handle network issues and long-running requests
boto_config = Config(
    read_timeout=300,  # Maximum time to wait for a response (5 minutes)
    connect_timeout=10,  # Maximum time to wait for connection establishment (10 seconds)
    retries={'max_attempts': 2}  # Number of retry attempts if the request fails
)

def ask_bedrock_converse(question, model_id="us.amazon.nova-premier-v1:0", endpoint="https://bedrock-runtime.us-east-1.amazonaws.com"):
    """
    Send a question to Bedrock using the non-streaming Converse API and get the complete response.

    Args:
        question (str): The question to ask the AI model
        model_id (str): The specific Bedrock model to use (default: Nova Premier)
        endpoint (str): The Bedrock API endpoint URL
    """
    try:
        # Create a Bedrock runtime client with our configuration
        # This client will handle communication with AWS Bedrock service
        session = boto3.Session(region_name='us-east-1',profile_name='nova-grounding-test-profile') # CHANGEME: Credential profile corresponding to allow-listed account and with bedrock access
        client = session.client("bedrock-runtime", region_name="us-east-1", endpoint_url=endpoint, config=boto_config)

        # Prepare the conversation in the format expected by Bedrock
        # This follows the conversational AI format with roles and content
        conversation = [
            {
                "role": "user",  # Indicates this message is from the user
                "content": [{"text": question}],  # The actual question text
            }
        ]

        # Configure tools that the AI model can use
        # In this case, we're enabling Nova Web Grounding functionality
        toolConfiguration = {
            "tools": [
                {
                    "systemTool": {
                        "name": "nova_grounding"  # Enables the model to search for real-time information
                    }
                }
            ]
        }

        # Print the question and formatting for better readability
        print(f"Question: {question}")
        print("=" * 50)  # Visual separator
        print("Response:")
        print("-" * 30)  # Another visual separator

        # Make the API call to Bedrock
        response = client.converse(
            modelId=model_id,  # Which AI model to use
            messages=conversation,  # The conversation history (just our question)
            toolConfig=toolConfiguration,
        )

        # Extract and display the request ID for debugging/tracking purposes
        request_id = response['ResponseMetadata']['RequestId']
        print(f"Request ID: {request_id}")
        print()

        # Simply print the response as JSON with nice formatting
        print(f"Response:\n{json.dumps(response['output']['message'], indent=4)}")
        print("-" * 80)
        
        # Return the response for use in other cells
        return response

    except Exception as e:
        # Handle any errors that occur during the API call or processing
        print(f"Error processing query: {e}")
        return None

## Step 5: Set the ModelId 
Change this to any supported model in the Nova family. Nova Premier is supported initially. 

In [ ]:
# Specify which AI model to use - you can change this to other available models. 

model_id = "us.amazon.nova-premier-v1:0"


## Step 6: Execute a query and format the response (Clean text with citations)
Below code will format the raw response to show clean text without thinking tags and with proper citations

In [ ]:
import re

def format_bedrock_response(response):
    """
    Format the Bedrock response to show clean text with citations
    
    Args:
        response: Raw Bedrock response dictionary
        
    Returns:
        Formatted text with inline citations and sources list
    """
    if not response or 'output' not in response:
        return "No response to format."
    
    message = response['output']['message']
    content = message.get('content', [])
    
    result_parts = []
    citation_counter = 1
    all_citations = {}
    
    # Process content items sequentially
    i = 0
    while i < len(content):
        item = content[i]
        
        # Handle text content
        if 'text' in item:
            text = item['text']
            
            # Skip tool use items
            if 'toolUse' in item:
                i += 1
                continue
            
            # Remove thinking tags completely
            text = re.sub(r'<thinking>.*?</thinking>', '', text, flags=re.DOTALL)
            text = re.sub(r'</?thinking[^>]*>', '', text)
            
            # Clean up the text
            text = text.strip()
            
            # Only process non-empty text
            if text:
                # Check if the next item is a citation for this text
                citation_added = False
                if i + 1 < len(content) and 'citationsContent' in content[i + 1]:
                    citation_item = content[i + 1]
                    citation_data = citation_item['citationsContent']
                    
                    if 'citations' in citation_data:
                        citations_for_this_text = []
                        for citation in citation_data['citations']:
                            if 'location' in citation and 'web' in citation['location']:
                                web_info = citation['location']['web']
                                url = web_info.get('url', '')
                                domain = web_info.get('domain', '')
                                
                                if url and domain:
                                    citations_for_this_text.append((domain, url))
                        
                        # Add citation numbers to this text
                        if citations_for_this_text:
                            for domain, url in citations_for_this_text:
                                text += f'[{citation_counter}]'
                                all_citations[citation_counter] = (domain, url)
                                citation_counter += 1
                            citation_added = True
                    
                    # Skip the citation item since we processed it
                    if citation_added:
                        i += 1
                
                result_parts.append(text)
        
        i += 1
    
    # Join all text parts with proper spacing
    result = ''.join(result_parts)
    
    # Clean up any double spaces or formatting issues
    result = re.sub(r'\s+', ' ', result)
    result = re.sub(r'\n\s*\n', '\n\n', result)
    
    # Add sources section if we have citations
    if all_citations:
        result += "\n\n**Sources:**\n"
        for num, (domain, url) in all_citations.items():
            result += f"{num}. {domain}: {url}\n"
    
    # Final cleanup
    result = re.sub(r'\n\n\n+', '\n\n', result)
    result = result.strip()
    
    return result

print("✅ Response formatting function defined successfully!")

In [ ]:
# Test with a question and format the response
question = "What are the latest developments in renewable energy technology?"
print(f"🔍 Testing with new question: {question}")
print("=" * 60)

# Get raw response
response = ask_bedrock_converse(question, model_id)

# Format and display
if response:
    print("\n📝 Formatted Response:")
    print("=" * 50)
    formatted = format_bedrock_response(response)
    print(formatted)
    print("=" * 50)
else:
    print("❌ Failed to get response for new question.")

In [ ]:
# Test with another question and format the response
new_question = "What are latest advancements in AI ?"
print(f"🔍 Testing with new question: {new_question}")
print("=" * 60)

# Get raw response
new_response = ask_bedrock_converse(new_question, model_id)

# Format and display
if new_response:
    print("\n📝 Formatted Response:")
    print("=" * 50)
    formatted_new = format_bedrock_response(new_response)
    print(formatted_new)
    print("=" * 50)
else:
    print("❌ Failed to get response for new question.")

## Step 7: Now let's stream the model response

We will use the Bedrock converse stream API to stream the model response. 

In [ ]:
import boto3
import time
from botocore.config import Config

# Configure timeout settings for the Bedrock API client
# These settings help handle network issues and long-running requests
boto_config = Config(
    read_timeout=300,  # Maximum time to wait for a response (5 minutes)
    connect_timeout=10,  # Maximum time to wait for connection establishment (10 seconds)
    retries={'max_attempts': 2}  # Number of retry attempts if the request fails
)

def stream_bedrock_response(question, model_id="us.amazon.nova-premier-v1:0", endpoint="https://bedrock-runtime.us-east-1.amazonaws.com"):
    """
    Stream response from Bedrock API and print chunks as they arrive

    Args:
        question (str): The question to ask the AI model
        model_id (str): The specific Bedrock model to use (default: Nova Premier)
        endpoint (str): The Bedrock API endpoint URL
    """
    try:
        # Create a Bedrock runtime client with our configuration
        # This client will handle communication with AWS Bedrock service
        session = boto3.Session(region_name='us-east-1',profile_name='nova-grounding-test-profile') # CHANGEME: Credential profile corresponding to allow-listed account and with bedrock access
        client = session.client("bedrock-runtime", region_name="us-east-1", endpoint_url=endpoint, config=boto_config)

        # Prepare the conversation in the format expected by Bedrock
        # This follows the conversational AI format with roles and content
        conversation = [
            {
                "role": "user",  # Indicates this message is from the user
                "content": [{"text": question}],  # The actual question text
            }
        ]

        # Configure tools that the AI model can use
        # In this case, we're enabling Nova Web Grounding functionality
        toolConfiguration = {
            "tools": [
                {
                    "systemTool": {
                        "name": "nova_grounding"  # Enables the model to search for real-time information
                    }
                }
            ]
        }

        # Print the question and formatting for better readability
        print(f"Question: {question}")
        print("=" * 50)  # Visual separator
        print("Streaming response:")
        print("-" * 30)  # Another visual separator

        # Make the streaming API call to Bedrock
        # converse_stream returns chunks of the response as they're generated
        streaming_response = client.converse_stream(
            modelId=model_id,  # Which AI model to use
            messages=conversation,  # The conversation history (just our question)
            toolConfig=toolConfiguration  # Tools the model can use
        )

        # Extract and display the request ID for debugging/tracking purposes
        request_id = streaming_response['ResponseMetadata']['RequestId']
        print(f"Request ID: {request_id}")
        print()

        # Process the streaming response chunk by chunk
        # This allows us to display text as it's being generated, not just at the end
        for chunk in streaming_response["stream"]:
            # Check if this chunk contains content (text, tool use, etc.)
            if "contentBlockDelta" in chunk:
                # Extract the actual content from the chunk
                delta = chunk["contentBlockDelta"]["delta"]

                # Handle different types of content
                if "text" in delta:
                    # Regular text content - print immediately without newline
                    # end="" prevents automatic newline, flush=True ensures immediate display
                    print(delta["text"], end="", flush=True)
                elif "toolUse" in delta:
                    # Model is using a tool (like nova_grounding) - show this activity
                    print(f"\n[TOOL_USE: {delta['toolUse']}]\n", end="", flush=True)
                elif "citation" in delta:
                    # Model is providing citations for its sources - display them
                    print(f"\n[CITATION: {delta['citation']}]\n", end="", flush=True)

        # Print completion message and formatting
        print("\n" + "=" * 50)
        print("Streaming complete!")

    except Exception as e:
        # Handle any errors that occur during the API call or processing
        print(f"Error processing query: {e}")

## Step 8: Try streaming with a query

In [ ]:
question = "What is today's market trends?"

# Specify which AI model to use - you can change this to other available models
# Nova Premier is Amazon's latest and most capable model
model_id = "us.amazon.nova-premier-v1:0"

# Call the streaming function with our question and model
stream_bedrock_response(question, model_id)